In [ ]:
# This code is used to overlap circle light curve on 
# KIC 9166862 AND other good light curves

In [ ]:
a = 10
a

In [ ]:
# Libraries to Load

import pytransit as trans
import numpy as np
import matplotlib.pyplot as plt
from corner import corner

In [ ]:
# Code to fit pytransit on Kepler LC

# TRANSIT.PY UCLA IAN

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 400
import numpy as np
import transit

In [ ]:
# Reproduce Figure 2 of Mandel & Agol (2002):

z = np.linspace(0, 1.25, 200)
gammavals = [[0.01, 0.01], [1., 0.], [0.85, 0.06]]
plt.figure()
for gammas in gammavals:
    f = transit.occultquad(z, 0.2, gammas)
    plt.plot(z, f)

In [ ]:
p, b = 0.1, 0.5
x = np.linspace(-1.25,1.25,100) #(arange(300.)/299. - 0.5)*2.
z = np.sqrt(x**2 + b**2)
gammas = [.75, .75]
F1, Funi, lambdad, etad = transit.occultquad(z, p, gammas, retall=True)

gammas = [.01, .01]
# F2 = 1. - ((1. - gammas[0] - 2.*gammas[1])*(1. - F1) + (gammas[0] + 2.*gammas[1])*(lambdad + 2./3.*(p > z)) + gammas[1]*etad) / (1. - gammas[0]/3. - gammas[1]/6.)
F2, Funi, lambdad, etad = transit.occultquad(z, p, gammas, retall=True)

plt.figure()
plt.plot(x, F1, x, F2)
plt.legend(['F1', 'F2'])

# KEPLER DATA LOAD

In [ ]:
kepid_array = np.load("../data/tfr_shallue/npy_shallue/kepid.npy")
print('kepid_load = ',kepid_array[0])

av_training_set_array = np.load("../data/tfr_shallue/npy_shallue/av_training_set.npy")
print('av_training_set_array = ',av_training_set_array[0])

local_view_array = np.load("../data/tfr_shallue/npy_shallue/local_view.npy")
local_view_array = local_view_array + 1
print('local_view_array = ',local_view_array[0])

In [ ]:
# Plot one light curve (local)
# plt.grid('on')
# plt.plot(np.arange(len(local_view_array[0])),local_view_array[0])
# plt.show()

In [ ]:
# Calculate SNR for each Local view lc and save them in an array
noise_array = np.zeros((len(local_view_array),120))
for i in np.arange(len(local_view_array)):
    noise_array[i][0:60] = local_view_array[i,0:60]
    noise_array[i][60:120] = local_view_array[i,141:202]
std_devs_Kepler = np.array([np.std(arr) for arr in noise_array])
SNR_Kepler = 1/std_devs_Kepler

In [ ]:
# Save the index of the light curve with SNR > 500
# and plot them
# select_kepler_lc = lc_np_array[SNR_Kepler>75]
# select_kepler_lc = select_kepler_lc + 1

SNR_Threshold = 1000
selected_kepler_index_mask = ((SNR_Kepler > SNR_Threshold) & (av_training_set_array==b'P')) 
selected_kepler_index = np.where(selected_kepler_index_mask)[0]
print(f"No. of light curves with SNR > {SNR_Threshold} = ",len(selected_kepler_index))
print(f"Total no. of light curves = ",len(SNR_Kepler))
# Plot the selected light curves 3 x 3
num = 3
fig,ax=plt.subplots(num,3, figsize=(12,6), gridspec_kw={ 'width_ratios': [1,1,1],
        'wspace': 0.2,'hspace': 0.4})

# ax[0][1].set_title('Shape',size=15)
# ax[0][0].set_title('Light Curve (Train Dataset)',size=15)
# ax[num-1][0].set_xlabel('Phase',size=13)
ph_kepler = np.linspace(-1,1,len(local_view_array[0]))


i = 0
for i in np.arange(0,num):
    # k = np.random.randint(0, len(select_kepler_lc)-50)
    k = int(np.random.choice(selected_kepler_index))
    ax[i][0].set_title(f'{np.round(kepid_array[k],0)} | SNR = {int(np.round(SNR_Kepler[k],0))} | {av_training_set_array[k]}',size=13)
    ax[i][0].set_ylim(-0.5,1.5)
    ax[i][0].scatter(ph_kepler, local_view_array[k],color = 'tab:red',s=2)
    ax[i][0].grid('on')

    k = int(np.random.choice(selected_kepler_index))
    ax[i][1].set_title(f'{np.round(kepid_array[k],0)} |SNR = {int(np.round(SNR_Kepler[k],0))} | {av_training_set_array[k]}',size=13)
    ax[i][1].set_ylim(-0.5,1.5)
    ax[i][1].scatter(ph_kepler, local_view_array[k],color = 'tab:red',s=2)
    ax[i][1].grid('on')

    k = int(np.random.choice(selected_kepler_index))
    ax[i][2].set_title(f'{np.round(kepid_array[k],0)} |SNR = {int(np.round(SNR_Kepler[k],0))} | {av_training_set_array[k]}',size=13)
    ax[i][2].set_ylim(-0.5,1.5)
    ax[i][2].scatter(ph_kepler, local_view_array[k],color = 'tab:red',s=2)
    ax[i][2].grid('on')
    i = i + 1

In [ ]:
# selecting left and right side by certain percentage
# (A) Kepler LC
# lc_np_array_offset_mask used to select the flat part by certain percentage
local_view_array_mask = np.copy(local_view_array)

for iter in np.arange(len(local_view_array_mask)):
    # 0.988 is working good | lower it and see changes # 0.96 - 0.97 -better # 0.95 -worse
    local_view_array_mask[iter][(local_view_array_mask[iter]>=0.98)] = 1.0
    local_view_array_mask[iter][(local_view_array_mask[iter]<0.98)] = 0.0

print("Length of one Kepler LC = ",len(local_view_array_mask[0]))



count_zeros_array_kepler = np.zeros((len(local_view_array),))
for iteration in np.arange(len(local_view_array)):
    # Calculate the number of occurrences of '0'
    count_zeros = np.count_nonzero(local_view_array_mask[iteration] == 0)  
    count_zeros_array_kepler[iteration] = count_zeros




In [ ]:
center_index_kepler = int(len(local_view_array[0])/2)
print("center_index_kepler = ",center_index_kepler)

# Create a figure and axis
fig, ax = plt.subplots()

for iteration in np.arange(1):
    # selected_kepler_index_elem = selected_kepler_index[iteration]
    selected_kepler_index_elem = np.random.choice(selected_kepler_index)
    print('selected_kepler_index_elem = ',selected_kepler_index_elem)
    left_index_kepler = int(center_index_kepler - int(count_zeros_array_kepler[selected_kepler_index_elem]/2)-int(count_zeros_array_kepler[selected_kepler_index_elem]/6)) 
    right_index_kepler = int(center_index_kepler + int(count_zeros_array_kepler[selected_kepler_index_elem]/2)+int(count_zeros_array_kepler[selected_kepler_index_elem]/6))
    # print("left_index = ",left_index)
    # print("right_index = ",right_index)
    lc_test_kepler=local_view_array[selected_kepler_index_elem][left_index_kepler:right_index_kepler+1]
    ax.scatter(np.linspace(-1,1,len(lc_test_kepler)),lc_test_kepler,s=5,label=f"KIC {kepid_array[selected_kepler_index_elem]}")
    # ax.plot(np.linspace(-1,1,len(lc_test)),lc_test)

plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.grid('on')
plt.show()


# TRAIN LC Load

In [ ]:
train_lc_dir =  '/scratch/abraham/Documents/mega_git/mega/data/train/npy/lc/lc_planet_oblate/lc_1_planet_oblate_circle.npy'
train_lc = np.load(train_lc_dir)


In [ ]:
# 2. Normalize the lightcurves
## - Train Set
train_lc_scaled = np.zeros(train_lc.shape)
for i in np.arange(len(train_lc_scaled)):
    train_lc_scaled[i] = (train_lc[i] - np.amin(train_lc[i]))/(np.amax(train_lc[i]) - np.amin(train_lc[i]))


In [ ]:
ph = np.linspace(-1,1,len(train_lc_scaled[0]))
plt.plot(ph,train_lc_scaled[0])
plt.grid('on')
plt.show()

In [ ]:
# (B) Simulated LC
# lc_np_array_offset_mask used to select the flat part by certain percentage
simul_lc_array_mask = np.copy(train_lc_scaled)

for iter in np.arange(len(simul_lc_array_mask)):
    # 0.988 is working good | lower it and see changes # 0.96 - 0.97 -better # 0.95 -worse
    simul_lc_array_mask[iter][(train_lc_scaled[iter]>=0.98)] = 1.0
    simul_lc_array_mask[iter][(train_lc_scaled[iter]<0.98)] = 0.0

print("Length of one simulated LC = ",len(simul_lc_array_mask[0]))

count_zeros_array_simul_LC = np.zeros((len(train_lc_scaled),))
for iteration in np.arange(len(train_lc_scaled)):
    # Calculate the number of occurrences of '0'
    count_zeros = np.count_nonzero(simul_lc_array_mask[iteration] == 0)  
    count_zeros_array_simul_LC[iteration] = count_zeros

In [ ]:
center_index_simulate = int(len(train_lc_scaled[0])/2)
print("center_index = ",center_index_simulate)

# Create a figure and axis
fig, ax = plt.subplots()

for iteration in np.arange(1):
    # Simulated LC
    selected_simulated_index = 4 # np.random.randint(0,len(train_lc_scaled))
    print('selected_simulated_index = ',selected_simulated_index)
    left_index_simul = int(center_index_simulate - int(count_zeros_array_simul_LC[selected_simulated_index]/2)-int(count_zeros_array_simul_LC[selected_simulated_index]/6)) 
    right_index_simul = int(center_index_simulate + int(count_zeros_array_simul_LC[selected_simulated_index]/2)+int(count_zeros_array_simul_LC[selected_simulated_index]/6))
    print("left_index_simul = ",left_index_simul) # 7 # 15 # 
    print("right_index_simul = ",right_index_simul)# 113 105 # 
    lc_test_simul=train_lc_scaled[selected_simulated_index][left_index_simul:right_index_simul+1]
    # ax.scatter(np.linspace(-1,1,len(lc_test)),lc_test,s=5,label=f"KIC {kepid_array[selected_kepler_index_elem]}")
    # ax.scatter(np.linspace(-1,1,len(lc_test_simul)),lc_test_simul,s=5)
    ax.plot(np.linspace(-1,1,len(lc_test_simul)),lc_test_simul,color='tab:red',label = 'Circle')

    # Kepler
    selected_kepler_index_elem = 5718 # np.random.choice(selected_kepler_index) # 14416 # 12096 # 
    print('selected_kepler_index_elem = ',selected_kepler_index_elem)
    print(f"KIC {int(kepid_array[selected_kepler_index_elem])}")
    left_index_kepler = int(center_index_kepler - int(count_zeros_array_kepler[selected_kepler_index_elem]/2)-int(count_zeros_array_kepler[selected_kepler_index_elem]/6)) 
    right_index_kepler = int(center_index_kepler + int(count_zeros_array_kepler[selected_kepler_index_elem]/2)+int(count_zeros_array_kepler[selected_kepler_index_elem]/6))
    print("left_index_kepler = ",left_index_kepler)
    print("right_index_kepler = ",right_index_kepler)
    # lc_test_kepler=local_view_array[selected_kepler_index_elem][left_index_kepler:right_index_kepler+1]
    lc_test_kepler=local_view_array[selected_kepler_index_elem][82:120] # [84:118]
    ax.scatter(np.linspace(-1,1,len(lc_test_kepler)),lc_test_kepler,s=5,label=f"KIC {int(kepid_array[selected_kepler_index_elem])}")

plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.grid('on')
plt.show()


# PYTRANSIT fit on Kepler

In [ ]:
k = int(np.random.choice(selected_kepler_index))
print("k = ",k)
print("KIC: ",int(np.round(kepid_array[k],0)))




In [ ]:
# Without cutting the lc
fit_this_k = k
print("KIC ID: ",int(np.round(kepid_array[fit_this_k],0)))
ph = np.linspace(-1,1,len(local_view_array[fit_this_k]))
Kepler_flux = local_view_array[fit_this_k]

plt.title(f"KIC {int(np.round(kepid_array[fit_this_k],0))}")
plt.plot(ph,Kepler_flux)
plt.show()

In [ ]:
# After cutting LC

fit_this_k = k
print("KIC ID: ",int(np.round(kepid_array[fit_this_k],0)))
ph = np.linspace(-1,1,len(local_view_array[fit_this_k]))
Kepler_flux = local_view_array[fit_this_k]

center_index_kepler = int(len(local_view_array[0])/2)
print('selected_kepler_index_elem = ',fit_this_k)
left_index_kepler = int(center_index_kepler - int(count_zeros_array_kepler[fit_this_k]/2)-int(count_zeros_array_kepler[fit_this_k]/6)) 
right_index_kepler = int(center_index_kepler + int(count_zeros_array_kepler[fit_this_k]/2)+int(count_zeros_array_kepler[fit_this_k]/6))


Kepler_flux_cut = local_view_array[fit_this_k][left_index_kepler:right_index_kepler+1]
ph_cut = np.linspace(-1,1,len(Kepler_flux_cut))

In [ ]:
plt.title(f"KIC {int(np.round(kepid_array[fit_this_k],0))}")
plt.plot(ph_cut,Kepler_flux_cut)
plt.show()

In [ ]:
ta = trans.TransitAnalysis(name='tutorial_1', passbands='Kepler', times=ph, fluxes=Kepler_flux)
# ta = trans.TransitAnalysis(name='tutorial_1', passbands='Kepler', times=ph_cut, fluxes=Kepler_flux_cut)

In [ ]:
ta.print_parameters(1)


In [ ]:
ta.set_prior('p_1', 'NP', 5, 1e-5)
ta.set_prior('k2_1', 'NP', 0.99, 1e-3)


In [ ]:
ta.print_parameters(1)


In [ ]:
ta.optimize_global(niter=500, npop=100)


In [ ]:
ta.plot_light_curves(method='fit');

In [ ]:
ta.sample_mcmc(niter=2000, thin=20, repeats=3, save=False)

In [ ]:
ta.save()


In [ ]:
df = ta.posterior_samples()

In [ ]:
ta.plot_light_curves(method='posterior');

In [ ]:
corner(df.posterior[['q1_Kepler', 'q2_Kepler']]);


In [ ]:
df